In [2]:
from flowstab.flow_stability import FlowStability
from flowstab.network_clustering import Clustering,avg_norm_var_information, run_multi_louvain
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc


df = pd.read_csv("Pre_processed_Data/primary_school_day2.csv")
scales = np.logspace(-7, 2, 100)   
real_taus = 1 / scales                

for index in range(10):
    taus=[i for x, i in enumerate(real_taus) if x%10==index]
    fs = FlowStability()
    fs.set_temporal_network(events_table=df, relabel_nodes=True)
    fs.compute_laplacian_matrices()
    fs.set_time_scale(taus)
    fs.compute_inter_transition_matrices()
    fs.time_direction = 0          
    fs.set_flow_clustering()
    fs._temporal_network.inter_T=None
    gc.collect()



    n_repetitions=200

    for tau in tqdm(taus):
        forward_partitions = {}
        clus_obj_for=fs.flow_clustering_forward[tau]
        _, cluster_lists, stabilities, _=run_multi_louvain(Clustering(p1=clus_obj_for.p1, p2=None,S=clus_obj_for.I_list[0]), num_repeat=n_repetitions)
        forward_partitions['clusters']=cluster_lists
        forward_partitions['stabilities']=stabilities
        np.save(f"Forward_partitions/forward_partitions-s={1/tau}.npy", forward_partitions, allow_pickle=True)
        fs.flow_clustering_forward[tau]=None
        del cluster_lists, stabilities,forward_partitions
        gc.collect()



    for tau in tqdm(taus):
        backward_partitions = {}
        clus_obj_back=fs.flow_clustering_backward[tau]
        _, cluster_lists, stabilities, _=run_multi_louvain(Clustering(p1=clus_obj_back.p1, p2=None,S=clus_obj_back.I_list[0]), num_repeat=n_repetitions)
        backward_partitions['clusters']=cluster_lists
        backward_partitions['stabilities']=stabilities
        np.save(f"Backward_partitions/backward_partitions-s={1/tau}.npy", backward_partitions, allow_pickle=True)
        fs.flow_clustering_backward[tau]=None
        del cluster_lists, stabilities,backward_partitions
        gc.collect()


    fs=None

2025-11-13 00:32:31,091 - INFO - flowstab\flow_stability.py:235 - PID:17720 - Set an empty temporal network.
2025-11-13 00:32:31,091 - INFO - flowstab\flow_stability.py:139 - PID:17720 - Successfully initiated a FlowStability instance!


2025-11-13 00:32:31,096 - INFO - flowstab\flow_stability.py:230 - PID:17720 - Set the temporal network: <class 'flowstab.temporal_network.ContTempNetwork'> with 238 nodes and 65150 events
2025-11-13 00:32:31,096 - INFO - flowstab\temporal_network.py:1176 - PID:17720 - Computing Laplacians
2025-11-13 00:32:31,168 - INFO - flowstab\temporal_network.py:1266 - PID:17720 - 0 over 1545
2025-11-13 00:32:31,168 - INFO - flowstab\temporal_network.py:1270 - PID:17720 - 0.00s
2025-11-13 00:32:56,496 - INFO - flowstab\temporal_network.py:1266 - PID:17720 - 1000 over 1545
2025-11-13 00:32:56,496 - INFO - flowstab\temporal_network.py:1270 - PID:17720 - 25.33s
2025-11-13 00:33:07,638 - INFO - flowstab\temporal_network.py:1326 - PID:17720 - Finished in 36.47523474693298
2025-11-13 00:33:07,638 - INFO - flowstab\flow_stability.py:530 - PID:17720 - Computing inter T matrices for time_scale=10000000.0.
2025-11-13 00:33:36,621 - INFO - flowstab\flow_stability.py:541 - PID:17720 - -> done.
2025-11-13 00:33